In [ ]:
# !pip install mediapipe

  Using cached mediapipe-0.10.32-py3-none-win_amd64.whl.metadata (9.8 kB)
Using cached mediapipe-0.10.32-py3-none-win_amd64.whl (10.2 MB)


**Level 1 (가벼운 경고)**: 
- 조건: 하품 1회 발생 OR 눈 1.5초 감김 1회 발생 OR 고개 떨굼 1회 발생
- 작동: 경고 알림음

**Level 2 (적극적 개입)**: 
- 조건: 최근 5분 이내에 Level 1 이벤트가 3회 누적되거나, 3분 이내 하품 2회 발생 시 OR 고개 떨굼 2회 발생 시.
- 작동: 날카로운 경고음, 음성 출력(예: "잠시 쉬었다가 운전하세요")

**Level 3 (긴급 솔루션 작동)**: 
- 조건: 눈을 3초 이상 감고 있거나, 최근 10분 이내에 Level 2가 2회 이상 발생했을 때.
- 작동: 최상위 졸음 방지 솔루션 가동. (문제 풀 때까지 지속되는 알림)

In [ ]:
from ultralytics import YOLO
import cv2
import time
import numpy as np
from utils.camPredictUtils import *
from utils.mediapipeUtils import *

model = YOLO("runs/detect/train_100k/weights/best_openvino_model")

# 웹캠 연결 (0번 카메라)
cap = cv2.VideoCapture(0)

# FaceProcessor 클래스 인스턴스 생성
mediapipeProcess = FaceProcessor()

try:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 프레임 좌우 반전
        frame = cv2.flip(frame, 1)
        color_frame = frame.copy()
        head_drop, bbox_coords, crop_coords = mediapipeProcess.process_frame(frame)
        cropped_frame = frame[crop_coords[1]:crop_coords[3], crop_coords[0]:crop_coords[2]]
        frame = cv2.cvtColor(cropped_frame, cv2.COLOR_BGR2GRAY)

        # cv2.imshow("YOLO Real-time Inference", frame)

        # 모델 추론 수행 (verbose=False로 로그 출력 억제)
        results = model.predict(
            source=frame,
            verbose=True,
            save=False,
            imgsz=640,
            conf=0.25,
            agnostic_nms=True,
            iou=0.3,
            classes=[0, 1, 2, 3],       # Face는 mediapipe로 추출
            device="intel:gpu"
            )
        
        # 결과 시각화 프레임 생성
        # annotated_frame = results[0].plot()
        filtered_data = filter_overlapping_parts(results)

        annotated_frame = draw_face_box(color_frame, bbox_coords, head_drop)
        annotated_frame = draw_filtered_results(color_frame, filtered_data, ['eye_opened', 'eye_closed', 'mouth_opened', 'mouth_closed'], crop_coords)
        
        # 실시간 시각화 창 표시
        cv2.imshow("YOLO Real-time Inference", annotated_frame)

        # 'q' 키를 누르면 루프 종료
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        # 0.2초 간격 대기
        time.sleep(0.3)
finally:
    # 자원 해제 및 윈도우 닫기
    cap.release()
    cv2.destroyAllWindows()

Loading runs/detect/train_100k/weights/best_openvino_model for OpenVINO inference...
Using OpenVINO LATENCY mode for batch=1 inference on GPU.0...

0: 640x640 3 eye_openeds, 33.3ms
Speed: 1.1ms preprocess, 33.3ms inference, 0.4ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 2 eye_openeds, 29.9ms
Speed: 1.8ms preprocess, 29.9ms inference, 0.3ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 2 eye_openeds, 1 mouth_closed, 30.3ms
Speed: 1.6ms preprocess, 30.3ms inference, 0.6ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 2 eye_openeds, 29.6ms
Speed: 1.1ms preprocess, 29.6ms inference, 0.5ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 1 mouth_closed, 29.5ms
Speed: 1.0ms preprocess, 29.5ms inference, 0.3ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 2 eye_openeds, 30.0ms
Speed: 0.9ms preprocess, 30.0ms inference, 0.3ms postprocess per image at shape (1, 1, 640, 640)

0: 640x640 4 eye_openeds, 1 mouth_closed, 29.7ms
